In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk

In [24]:
reviews_df = pd.read_csv("GameReviewsFinal.csv")
stats_df = pd.read_csv("GameStatsFinal.csv")

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (2,3,6,7,8,9,10,11,12,13,14,15,16,17,18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (0,1,5,6,7,8,10,11,12,13,14,15,16,17,21,22,24,25,26,27) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [46]:
reviews_df.head(10)
reviews_df.shape

# First two columns look like indexes so I'll be removing them
# Also, dont think we need recommendation id
# author may be useful but not sure currently so removing it
# leaving the other columns

(302593, 19)

In [44]:
my_reviews = reviews_df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'recommendationid', 'author'], axis=1)
my_reviews.head()

,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,timestamp_dev_responded,developer_response,appid
0,english,"Unfortunately, at this price point I was expec...",1572942648,1572942648,False,11,2,0.658271,0,True,False,True,1.60828e+09,This! Completely fair review! Santa's Visit la...,1155810
1,english,A cute sandbox experience. Fun for the Holiday...,1608896024,1608896024,True,7,0,0.599486,0,True,False,True,NaN,NaN,1155810
2,english,While its a very short experience it has fanta...,1606426930,1606426930,True,7,0,0.571652,0,True,False,True,NaN,NaN,1155810
3,english,I got Santa trapped in Purgatory.\n10/10 holid...,1577329940,1577329940,True,7,1,0.571652,0,True,False,True,NaN,NaN,1155810
4,english,[h1]Experienced on the Oculus Quest 2[/h1]\n\n...,1640408475,1640408475,True,3,1,0.566025,0,True,False,True,NaN,NaN,1155810


In [45]:
my_reviews.isna().sum()

# Out of 302593 total reviews, 252824 are missing the appid column. 
# We can not use these for more detailed analysis

language                            0
review                            140
timestamp_created                   0
timestamp_updated                   0
voted_up                            0
votes_up                            0
votes_funny                         0
weighted_vote_score                 0
comment_count                       0
steam_purchase                      0
received_for_free                   0
written_during_early_access         0
timestamp_dev_responded         39351
developer_response             292175
appid                          252824
dtype: int64

In [47]:
# check for missing reviews and removing them

subset = ['review', 'appid']
my_reviews.dropna(subset=subset, axis=0, inplace=True)

In [48]:
# Converting the review column to string
my_reviews['review'] = my_reviews['review'].astype('str')
my_reviews['language'] = my_reviews['language'].astype('str')

In [49]:
my_reviews.info()
my_reviews.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49745 entries, 0 to 302561
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   language                     49745 non-null  object
 1   review                       49745 non-null  object
 2   timestamp_created            49745 non-null  object
 3   timestamp_updated            49745 non-null  object
 4   voted_up                     49745 non-null  object
 5   votes_up                     49745 non-null  object
 6   votes_funny                  49745 non-null  object
 7   weighted_vote_score          49745 non-null  object
 8   comment_count                49745 non-null  object
 9   steam_purchase               49745 non-null  object
 10  received_for_free            49745 non-null  object
 11  written_during_early_access  49745 non-null  object
 12  timestamp_dev_responded      10417 non-null  object
 13  developer_response           1

,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,timestamp_dev_responded,developer_response,appid
0,english,"Unfortunately, at this price point I was expec...",1572942648,1572942648,False,11,2,0.658271,0,True,False,True,1.60828e+09,This! Completely fair review! Santa's Visit la...,1155810
1,english,A cute sandbox experience. Fun for the Holiday...,1608896024,1608896024,True,7,0,0.599486,0,True,False,True,NaN,NaN,1155810
2,english,While its a very short experience it has fanta...,1606426930,1606426930,True,7,0,0.571652,0,True,False,True,NaN,NaN,1155810
3,english,I got Santa trapped in Purgatory.\n10/10 holid...,1577329940,1577329940,True,7,1,0.571652,0,True,False,True,NaN,NaN,1155810
4,english,[h1]Experienced on the Oculus Quest 2[/h1]\n\n...,1640408475,1640408475,True,3,1,0.566025,0,True,False,True,NaN,NaN,1155810


In [33]:
# stats_df.head(10)

In [ ]:
review_length = my_reviews['review'].str.len()
print(my_reviews.mean())
print(my_reviews.std())
print(my_reviews.max())
print(my_reviews.min())

In [ ]:
# Strings with length 1 don't make any sense. Lets check some strings with length less than 10

# First I'll add review_length as a column
my_reviews['review_length'] = review_length


In [41]:
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS

In [43]:
my_pattern = r'\b[^\d\W][^\d\W]+\b'
vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=50, token_pattern=my_pattern, stop_words=ENGLISH_STOP_WORDS)\
.fit(reviews_df.review.values.astype('str'))

In [8]:
X_txt = vectorizer.transform(reviews_df.review.values.astype('str'))

In [9]:
X = pd.DataFrame(X_txt.toarray(), columns=vectorizer.get_feature_names())
print('Top 5 rows of the DataFrame: ', X.head())

Top 5 rows of the DataFrame:          bad  better       bit  character  different  don  easy  feel  \
0  0.000000     0.0  0.000000        0.0        0.0  0.0   0.0   0.0   
1  0.000000     0.0  0.467513        0.0        0.0  0.0   0.0   0.0   
2  0.000000     0.0  0.000000        0.0        0.0  0.0   0.0   0.0   
3  0.000000     0.0  0.000000        0.0        0.0  0.0   0.0   0.0   
4  0.187449     0.0  0.000000        0.0        0.0  0.0   0.0   0.0   

        fun      game  ...  story    things    think  time  use   ve  want  \
0  0.000000  0.108531  ...    0.0  0.562994  0.00000   0.0  0.0  0.0   0.0   
1  0.699941  0.190865  ...    0.0  0.000000  0.00000   0.0  0.0  0.0   0.0   
2  0.000000  0.000000  ...    0.0  0.000000  0.00000   0.0  0.0  0.0   0.0   
3  0.000000  0.000000  ...    0.0  0.000000  0.00000   0.0  0.0  0.0   0.0   
4  0.000000  0.375142  ...    0.0  0.000000  0.18524   0.0  0.0  0.0   0.0   

        way  work     worth  
0  0.245900   0.0  0.000000  
1  0.000